# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

C:\Users\I822363\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\I822363\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
int_num_test = int(num_test/2)
test_data, test_labels = newsgroups_test.data[int_num_test:], newsgroups_test.target[int_num_test:]
dev_data, dev_labels = newsgroups_test.data[:int_num_test], newsgroups_test.target[:int_num_test]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print ('training label shape:', train_labels.shape)
print ('test label shape:', test_labels.shape)
print ('dev label shape:', dev_labels.shape)
print ('labels names:', newsgroups_train.target_names)

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


In [3]:
np.unique(train_labels[0:5])

array([0, 1, 2, 3], dtype=int64)

(1) For each of the first 5 training examples, print the text of the message along with the label.

In [4]:
def P1(num_examples=5):
### STUDENT START ###
    for lv_counter in range(num_examples):
        print("\n\n",train_data[lv_counter], "\ntraining label : ", train_labels[lv_counter])

### STUDENT END ###
P1()




 Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych 
training label :  1


 

Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have been demonstrating such evil corruption
for centuries. 
training label :  3


 
 >In arti

## (2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

In [5]:
#def P2():
### STUDENT START ###
vectorizer = CountVectorizer()
corpus = train_data
X = vectorizer.fit_transform(corpus)
X 


### STUDENT END ###
#P2()

<2034x26879 sparse matrix of type '<class 'numpy.int64'>'
	with 196700 stored elements in Compressed Sparse Row format>

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

In [6]:
from scipy.sparse import csr_matrix

# average no of features that are non zero per example
print('average no of features that are non zero per example: ',round(X.getnnz()/X.shape[0]))

# fraction of matrix entries that are non-zero
print('fraction of matrix entries that are non-zero:', X.getnnz()/np.prod(X.shape))


#A = csr_matrix([[1,2,0],[0,0,3],[4,0,5],[1,1,1],[0,0,0]])



average no of features that are non zero per example:  97
fraction of matrix entries that are non-zero: 0.00359782722696


> **_On an average, there are 97 non-zero features per sample and 0.36% of entries that are non-zero!_**

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.



In [7]:
analyze = vectorizer.build_analyzer()
print('0th feature name:',vectorizer.get_feature_names()[0])
print('last feature name:',vectorizer.get_feature_names()[-1])



0th feature name: 00
last feature name: zyxel


c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?


In [8]:
vectorizer2 = CountVectorizer(vocabulary = ["atheism", "graphics", "space", "religion"])
X2 = vectorizer2.fit_transform(corpus)
analyze2 = vectorizer2.build_analyzer()
print('feature name:',vectorizer2.get_feature_names()[-4:])
print('Training vector shape no of rows: %d , no of features %d' % X2.shape)

# average no of features that are non zero per example
print('average no of features that are non zero: ',round(X2.getnnz()/X2.shape[0],2))


feature name: ['atheism', 'graphics', 'space', 'religion']
Training vector shape no of rows: 2034 , no of features 4
average no of features that are non zero:  0.27


> **_0.27 out of 4 features is 6.75%. Roughly 6.75% of the features is non zero_**

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?


In [9]:
bitrigram_vectorizer = CountVectorizer(ngram_range=(2,3),
#                    token_pattern=r'\b\w+\b', 
    min_df=1)
X_3 = bitrigram_vectorizer.fit_transform(corpus).toarray()
print('Vocabulary shape for bigram and trigram features: (rows, columns)', X_3.shape)
#analyze3 = bigram_vectorizer.build_analyzer()
#analyze3('Bi-grams are cool!') == (
#        ['bi', 'grams', 'are', 'cool', 'bi grams', 'grams are', 'are cool'])

# print first 100 bigram and trigram features
print('Some Example features: \n',bitrigram_vectorizer.get_feature_names()[:20])



Vocabulary shape for bigram and trigram features: (rows, columns) (2034, 510583)
Some Example features: 
 ['00 10', '00 10 followed', '00 10 opening', '00 12', '00 12 00', '00 13', '00 13 phone', '00 21', '00 21 pdt', '00 22', '00 22 fax', '00 30', '00 30 afternoon', '00 30 lunch', '00 47', '00 47 apr', '00 50', '00 50 worth', '00 52', '00 52 01']


**Question: Why is analyzer necessary?**

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?


In [10]:
vectorizer = CountVectorizer(min_df=10)
X = vectorizer.fit_transform(corpus)
print('Vocabulary shape:', X.shape[1]) 


Vocabulary shape: 3064


f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

In [11]:
dev_vectorizer = CountVectorizer()
dev_X = dev_vectorizer.fit_transform(dev_data)


train_vectorizer = CountVectorizer()
train_X = train_vectorizer.fit_transform(train_data)

dev_features = dev_vectorizer.get_feature_names()
train_features = train_vectorizer.get_feature_names()

# Dev features that are not in train data
len(list(set(dev_features) - set(train_features)))


4027

> **_There are 4037 words or features present in Dev Data and not present in the training data_**

**Question: Is it train-dev or dev - train?**

(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

Let's do grid search for each of the 3 algorithms and get the best parameters:

In [14]:
# Get best parameters for KNN
knn = KNeighborsClassifier(algorithm = 'brute')
parameters = [{'weights': ['uniform', 'distance'], 'n_neighbors': [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}]
grid_kNN = GridSearchCV(knn, parameters, cv=10)
grid_kNN.fit(X_train,y_train)
print("\n best parameters:", grid_kNN.best_params_)
print("\n best score:", grid_kNN.best_score_)
grid_kNN.score(X_dev, y_dev)

#Get best parameters for Logistic Regression

#Get best parameters for Logistic Regression



 best parameters: {'n_neighbors': 70, 'weights': 'uniform'}

 best score: 0.4582104228121927


0.4526627218934911

In [15]:
#def P3():
### STUDENT START ###

### STUDENT END ###
#P3()

import sys
from time import time
import logging
import numpy as np

print('data loaded')

# order of labels in `target_names` can be different from `categories`
target_names = newsgroups_train.target_names

def size_mb(docs):
    return sum(len(s.encode('utf-8')) for s in docs) / 1e6

train_data_size_mb = size_mb(train_data)
dev_data_size_mb = size_mb(dev_data)

print("%d documents - %0.3fMB (training set)" % (
    len(train_data), train_data_size_mb))
print("%d documents - %0.3fMB (test set)" % (
    len(dev_data), dev_data_size_mb))
print("%d categories" % len(categories))
print()

# split a training set and a test set
#train_labels, dev_labels = train_data.target, data_test.target

print("Extracting features from the training data using a sparse vectorizer")
t0 = time()

vectorizer = CountVectorizer( 
#                              max_df=0.5
                              )
X_train = vectorizer.fit_transform(train_data)
y_train = train_labels
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, train_data_size_mb / duration))
print("n_samples: %d, n_features: %d" % X_train.shape)
print()


print("Extracting features from the test data using the same vectorizer")
t0 = time()
X_dev = vectorizer.transform(dev_data)
y_dev = dev_labels
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, dev_data_size_mb / duration))
print("n_samples: %d, n_features: %d" % X_dev.shape)
print()

# mapping from integer feature name to original token string
feature_names = vectorizer.get_feature_names()

if feature_names:
    feature_names = np.asarray(feature_names)


def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."

# #############################################################################
# Benchmark classifiers
def benchmark(clf, ipts_print_report='', ipts_print_cm=''):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_dev)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(y_dev, pred)
    print("accuracy:   %0.3f" % score)

    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))

        if opts.print_top10 and feature_names is not None:
            print("top 10 keywords per class:")
            for i, label in enumerate(target_names):
                top10 = np.argsort(clf.coef_[i])[-10:]
                print(trim("%s: %s" % (label, " ".join(feature_names[top10]))))
        print()

    if ipts_print_report:
        print("classification report:")
        print(metrics.classification_report(y_dev, pred,
                                            target_names=target_names))

    if ipts_print_cm:
        print("confusion matrix:")
        print(metrics.confusion_matrix(y_dev, pred))

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time



# Routine that looks at neighbors ranging from 10 to 1000 for all dev data
# and determines the k value that has the highest accuracy
def get_best_k_in_kNN():
    k_result = []
    for n in range(10,1000,10):
        clf = KNeighborsClassifier(n_neighbors=n)
        clf.fit(X_train, y_train)
        preds = clf.predict(X_dev)
        accuracy = np.where((preds == y_dev),1,0).sum() / y_dev.shape[0]
        k_result.append([accuracy,n])
        k_result_list = pd.DataFrame(k_result, columns=["accuracy", "n"])
    # assign and return best K value        
    best_k_value = k_result_list.loc[k_result_list['accuracy'].idxmax()]["n"]
    return int(best_k_value)

results = []
for clf, name in (
        (KNeighborsClassifier(n_neighbors=get_best_k_in_kNN()), "kNN"),
        ):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf, ipts_print_report='Y'))
print('Results = \n')
print(results)


data loaded
2034 documents - 2.428MB (training set)
676 documents - 1.013MB (test set)
4 categories

Extracting features from the training data using a sparse vectorizer
done in 0.540378s at 4.493MB/s
n_samples: 2034, n_features: 26879

Extracting features from the test data using the same vectorizer
done in 0.153106s at 6.615MB/s
n_samples: 676, n_features: 26879

kNN
________________________________________________________________________________
Training: 
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=110, p=2,
           weights='uniform')
train time: 0.001s
test time:  0.119s
accuracy:   0.479
classification report:
                    precision    recall  f1-score   support

       alt.atheism       0.50      0.33      0.39       165
     comp.graphics       0.55      0.62      0.58       185
         sci.space       0.48      0.57      0.52       199
talk.religion.misc       0.35      0.33      0.34 

**ANSWER:** 
> a. Why doesn't nearest neighbors work well for this problem?

> _Approximately 99% of the training data is zero. From the first question, we know that for the 1st sample in the training data, there are only **97 out of the 34118** features that are non zero. Because of the **sparseness** and that kNN suffers from the curse of high dimensionality, the accuracy is not great._

> _b. Any ideas why logistic regression doesn't work as well as Naive Bayes?_

> _ No_

> _
_

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

In [ ]:
#def P4():
### STUDENT START ###

### STUDENT END ###
#P4()

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

In [ ]:
#def empty_preprocessor(s):
#    return s

#def better_preprocessor(s):
### STUDENT START ###

### STUDENT END ###

#def P5():
### STUDENT START ###

### STUDENT END ###
#P5()

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

In [ ]:
#def P6():
    # Keep this random seed here to make comparison easier.
    #np.random.seed(0)

    ### STUDENT START ###
    
    ### STUDENT END ###
#P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

In [ ]:
#def P7():
### STUDENT START ###

## STUDENT END ###
#P7()

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.